## [MAUVE & ROUGE] ASQA & ELI5

In [ ]:
import copy
import json
from metrics import load_file

# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/eli5_sonnet_test_base.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/asqa_test_output_sonnet_sonnet.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/eli5_test_output_sonnet_sonnet.json"
# with open("/shared/eng/pj20/firas_data/test_datasets/results/asqa_sonnet_retrieval_top_5_base.json", "r") as f:
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_asqa_answerer_test_results_all_graph_no_ret.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_asqa_answerer_test_results_no_graph.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_asqa_answerer_test_results_all_graph_no_gtoken.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/eli5_sonnet_retrieval_top_5_sure_200.jsonl"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_eli5_answerer_test_results_all_graph_3_20.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_asqa_answerer_test_results_all_graph_3_20_8b.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_asqa_answerer_test_results_all_graph_5_20_8b.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_eli5_answerer_test_results_all_graph_7_20_8b.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_asqa_answerer_test_results_all_graph_10_20_8b.json"
file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_asqa_answerer_test_results_graph_0.5_8b.json"

if "jsonl" in file_path:
    data = load_file(file_path)
else:
    with open(file_path, "r") as f:
        data_ = json.load(f)


# new_data = []
# if "test_output" in file_path:
#     for i in range(len(data['output'])):
#         new_data.append({"output": data['output'][i], "answer": data['answer'][i], "question": data['question'][i]})
#     data = new_data

# else:
#     data = data['data']
#     if "eli5" in file_path:
#         for item in data:
#             item['answer'] = item['answers']

data = []
if "graphllm" in file_path:
    for item in data_:
        data.append({"question": item['input'].split("[Long Form] Question: ")[1].lower(), "output": item['prediction'].lower(), "answer": item['label'].lower()})
            


In [ ]:
import os
from metrics import *

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

normalized_data = copy.deepcopy(data)
# for i in range(len(normalized_data)):
#     normalized_data[i]['output'] = remove_citations(normalized_data[i]['output'])

references = [' '.join((item['question'] + " " + item['answer'].strip()).split()[:100]).rstrip(string.punctuation) for item in normalized_data]
predictions = [' '.join((item['question'] + " " + item['output'].strip()).split()[:100]).rstrip(string.punctuation) for item in normalized_data]


print("ROUGE: ", compute_rouge(normalized_data))
print("MAUVE: ", mauve_score(predictions, references))


## PopQA, TriviaQA, PubHealth, ARC-C, 2WikiMultiHop

In [ ]:
import copy
import json
from metrics import *
from utils import *

# data = load_file("/shared/eng/pj20/firas_data/test_datasets/results/popqa_sonnet_retrieval_top_5.jsonl")
# with open("/shared/eng/pj20/firas_data/test_datasets/pubhealth_test_output_sonnet_sonnet.json", "r") as f:
# with open("/shared/eng/pj20/firas_data/test_datasets/arc_c_test_output_sonnet_sonnet.json", "r") as f:
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_triviaqa_answerer_test_results.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_pubhealth_answerer_test_results_all_graph.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_pubhealth_answerer_test_results.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/triviaqa_test_output_sonnet_sonnet.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_popqa_answerer_test_results_all_graph.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_triviaqa_answerer_test_results_all_graph.json"

# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_arc_c_answerer_test_results_all_graph.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/2wikimultihop_test_output_llama2-7b_sonnet_v3.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_2wikimultihop_answerer_test_results_all_graph_both.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_triviaqa_answerer_test_results_all_graph_ptuned.json"

# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_pubhealth_answerer_test_results_all_graph_both.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_pubhealth_answerer_test_results_all_graph_ptuned.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_arc_c_answerer_test_results_all_graph_both.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_pubhealth_answerer_test_results_all_graph_no_gtoken.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_pubhealth_answerer_test_results_no_graph.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_triviaqa_answerer_test_results_no_graph.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_2wikimultihop_answerer_test_results_no_graph.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_triviaqa_answerer_test_results_all_graph_no_gtoken.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_2wikimultihop_answerer_test_results_all_graph_no_gtoken.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_triviaqa_answerer_test_results_all_graph_no_plan.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_2wikimultihop_answerer_test_results_all_graph_no_plan.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_2wikimultihop_answerer_test_results_all_graph_both.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/popqa_sonnet_retrieval_top_5_sure_200.jsonl"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/pubhealth_sonnet_retrieval_top_5_sure_200.jsonl"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/arc_c_sonnet_retrieval_top_5_sure_200.jsonl"

# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_pubhealth_answerer_test_results_all_graph_3_20_8b.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_triviaqa_answerer_test_results_all_graph_5_20_8b.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_pubhealth_answerer_test_results_all_graph_1_20_8b.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_2wikimultihop_answerer_test_results_all_graph_5_20_8b.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_pubhealth_answerer_test_results_all_graph_5_20.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_pubhealth_answerer_test_results_all_graph_7_20_8b.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_triviaqa_answerer_test_results_all_graph_7_20.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_2wikimultihop_answerer_test_results_all_graph_7_20.json"
# file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_2wikimultihop_answerer_test_results_all_graph_10_20_8b.json"
# 
file_path = "/shared/eng/pj20/firas_data/test_datasets/results/graphllm_triviaqa_answerer_test_results_graph_0.7_8b.json"

if "jsonl" in file_path:
    data = load_file(file_path)
else:
    with open(file_path, "r") as f:
        data_ = json.load(f)

    data = []
    if "graphllm" in file_path:
        print("true")
        if "2wikimultihop" not in file_path and "pubhealth" not in file_path and "arc_c" not in file_path:
            for item in data_:
                data.append({"output": item['prediction'].lower(), "golds": [d.lower() for d in item['label']]})
        elif "pubhealth" in file_path or "arc_c" in file_path:
            for item in data_:
                data.append({"output": item['prediction'].lower(), "golds": item['label'].lower()})
        else:
            for item in data_:
                data.append({"output": item['prediction'].lower(), "golds": [item['label'].lower()]})

    elif "2wikimultihop" in file_path:
        for i in range(len(data_['output'])):
            data.append({"output": data_['output'][i].lower(), "golds": [data_['answer'][i].lower()]})
    else:
        for i in range(len(data_['output'])):
            data.append({"output": data_['output'][i].lower(), "golds": [d.lower() for d in data_['answer'][i]]})



In [ ]:
data[1]

In [76]:
# data = [d for d in data if d["output"] == "true" or d["output"] == "false"]

In [ ]:
from tqdm import tqdm

# golds_mapping = {
#     "1": "a",
#     "2": "b",
#     "3": "c",
#     "4": "d",
#     "a": "a",
#     "b": "b",
#     "c": "c",
#     "d": "d"
# }


# data_["metric_result"] = []
metric_result_1 = []
for i in tqdm(range(len(data))):
    # pubhealth
    # data_["metric_result"].append(accuracy(data[i]["output"], data[i]["golds"]))
    # triviaqa, popqa, arc-c
    # data_["metric_result"].append(match(data[i]["output"], data[i]["golds"]))
    # if data_["metric_result"][i] == 1:
    #     data_["answer"][i].append(data[i]["output"])
    # result, add = cosine_match(data[i]["output"], data[i]["golds"])
    
    # result= match(data[i]["output"], golds_mapping[data[i]["golds"][0]])
    if "apolo" in data[i]["output"].lower() or "the information" in data[i]["output"].lower():
        continue
    
    # result, add= cosine_match(data[i]["output"], data[i]["golds"])
    # result = f1_score(data[i]["output"], data[i]["golds"])
    # result = accuracy(data[i]["output"], data[i]["golds"])
    result = match(data[i]["output"], data[i]["golds"])
    
    metric_result_1.append(result)
    # if add:
    #     data_[i]['label'].append(data[i]["output"])
    #     print(f"Added: {data[i]['output']} to {data_[i]['label']}")
    # if add:
    #     data_['answer'][i].append(data[i]["output"])
    #     print(f"Added: {data[i]['output']} to {data_['answer'][i]}")
    
# print("Match: ", np.mean(data_["metric_result"]))
print("Match: ", np.mean(metric_result_1))